In [ ]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import layers
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, DepthwiseConv2D, SeparableConv2D, GlobalAveragePooling2D
from keras import optimizers
from keras.regularizers import l2
import numpy as np
from keras.callbacks import ReduceLROnPlateau, CSVLogger
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import files

In [ ]:


class OptimizedCIFAR10CNN:
    def __init__(self, lr, train=True):
        self.num_classes = 10
        self.weight_decay = 1e-4
        self.x_shape = [32, 32, 3]
        self.lr = lr

        self.tiny_vgg = self.build_model()
        if train:
            self.tiny_vgg, self.history = self.train(self.tiny_vgg)
        else:
            self.tiny_vgg.load_model('TinyVGG.keras')

    def plot_training_history(self, history):
        plt.figure(figsize=(12, 6))

        plt.subplot(1, 2, 1)
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('Model Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend(['Train', 'Validation'], loc='upper right')

        plt.subplot(1, 2, 2)
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('Model Accuracy')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')
        plt.legend(['Train', 'Validation'], loc='lower right')

        plt.tight_layout()
        plt.show()


    def build_model(self):
        weight_decay = self.weight_decay

        model = keras.Sequential([
            # 1st Block
            layers.Conv2D(64, (3, 3), padding='same', kernel_regularizer=keras.regularizers.l2(weight_decay), input_shape=self.x_shape, activation='relu', name="conv2d_1"),
            layers.BatchNormalization(name="batch_normalization_1"),
            layers.Dropout(0.3, name="dropout_1"),
            layers.Conv2D(64, (3, 3), padding='same', kernel_regularizer=keras.regularizers.l2(weight_decay), activation='relu', name="conv2d_2"),
            layers.BatchNormalization(name="batch_normalization_2"),
            layers.MaxPooling2D(pool_size=(2, 2), name="max_pooling2d_1"),

            # 2nd Block
            layers.Conv2D(128, (3, 3), padding='same', kernel_regularizer=keras.regularizers.l2(weight_decay), activation='relu', name="conv2d_3"),
            layers.BatchNormalization(name="batch_normalization_3"),
            layers.Dropout(0.4, name="dropout_2"),
            layers.Conv2D(128, (3, 3), padding='same', kernel_regularizer=keras.regularizers.l2(weight_decay), activation='relu', name="conv2d_4"),
            layers.BatchNormalization(name="batch_normalization_4"),
            layers.MaxPooling2D(pool_size=(2, 2), name="max_pooling2d_2"),

            # 3rdBlock
            layers.Conv2D(128, (3, 3), padding='same', kernel_regularizer=keras.regularizers.l2(weight_decay), activation='relu', name="conv2d_5"),
            layers.BatchNormalization(name="batch_normalization_5"),
            layers.Conv2D(128, (3, 3), padding='same', kernel_regularizer=keras.regularizers.l2(weight_decay), activation='relu', name="conv2d_6"),
            layers.BatchNormalization(name="batch_normalization_6"),
            layers.MaxPooling2D(pool_size=(2, 2), name="max_pooling2d_3"),
            layers.Dropout(0.4, name="dropout_3"),

            # Fully Connected Layers
            layers.Flatten(name="flatten_1"),
            layers.Dense(256, kernel_regularizer=keras.regularizers.l2(weight_decay), activation='relu', name="dense_1"),
            layers.BatchNormalization(name="batch_normalization_7"),
            layers.Dropout(0.5, name="dropout_4"),
            layers.Dense(self.num_classes, activation='softmax', name="dense_2")
        ])

        return model


    def normalize(self,X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7)
        return X_train, X_test

    def normalize_production(self,x):
        mean = 120.707
        std = 64.15
        return (x-mean)/(std+1e-7)

    def predict(self,x,normalize=True,batch_size=50):
        if normalize:
            x = self.normalize_production(x)
        return self.tiny_vgg.predict(x,batch_size)


    def train(self, model):
        model_name = 'TinyVGG'
        batch_size = 128
        max_epochs = 150
        learning_rate = self.lr

        (x_train, y_train), (x_test, y_test) = cifar10.load_data()
        x_train = x_train.astype('float32')
        x_test = x_test.astype('float32')
        x_train, x_test = self.normalize(x_train, x_test)

        y_train = keras.utils.to_categorical(y_train, self.num_classes)
        y_test = keras.utils.to_categorical(y_test, self.num_classes)

        reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                      factor=0.7,
                                      patience=3,
                                      min_lr = 0.00001)


        datagen = ImageDataGenerator(
              brightness_range=[0.4,1.8],
              fill_mode='nearest',
              rotation_range=10,
              width_shift_range=0.1,
              height_shift_range=0.1,
              horizontal_flip=True
        )
        datagen.fit(x_train)

        model.compile(loss='categorical_crossentropy', optimizer=optimizers.legacy.Adam(learning_rate = learning_rate), metrics=['accuracy'])

        csv_logger = CSVLogger(f"{model_name}_history.csv")

        history = model.fit(datagen.flow(x_train, y_train, batch_size = batch_size),
                                      steps_per_epoch = x_train.shape[0] // batch_size,
                                      epochs = max_epochs,
                                      validation_data = (x_test, y_test),
                                      callbacks = [csv_logger, reduce_lr],
                                      verbose = 2)

        # Plot training history
        self.plot_training_history(history)

        model.save(f"{model_name}.keras")
        return model, history

In [ ]:

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# Train the model
trained_model = OptimizedCIFAR10CNN(lr = 0.001)

tiny_vgg_model = trained_model.tiny_vgg
tiny_vgg_history = trained_model.history

In [ ]:
cifar10_classes = ["Airplane", "Automobile", "Bird", "Cat", "Deer", "Dog", "Frog", "Horse", "Ship", "Truck"]

In [ ]:
y_test_labels = y_test.argmax(axis=1)  # Convert one-hot to labels
predictions = tiny_vgg_model.predict(x_test).argmax(axis=1)  # Get predicted labels